In [1]:
import pandas as pd
import datetime
import datetime
import re
import lxml
from bs4 import BeautifulSoup
import json
from calendar import monthrange

In [2]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")
#date_str = '2018-02-26'


#js = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_calendar_' + date_str + '.json')

js = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/single_cal_test.json')

js.head()

,calendar,ext_at,property_id,review_count,review_value
0,"<thead><tr><th scope=""col""><span title=""Monday...",2018-03-02 22:27:27,9064241,192,4.9


In [29]:
d = datetime.date.today()
mr = monthrange(d.year, d.month )

cur_month = d.month
cur_year = d.year

first_page = True



fp = open('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar_' + date_str + '.json', 'w')


for index, row in js.iterrows():   
    
    all_months = {}
    
    cur_month = d.month
    print("Current month: " + str(cur_month))
    

    raw_cal = "<html><body><table>" + row['calendar'] + "</table></body></html>"
    
    soup = BeautifulSoup(raw_cal, 'lxml') # Parse the HTML as a string
    soup_tables = soup.find_all('tbody')# Grab the first table
    
    month_tb_count = 1

    for month_tb in soup_tables:
 
        table = soup.find("table")
        
        cells = table.findAll("td")
        
        print("Month_tb: "+ str(month_tb_count))
        
        print("Length of cells:" + str(len(cells)))
            
        for td in cells:
            
            #print("TD: " + str(td))

            try:
                day_val = int(td.text)
                
                date_full = datetime.date(cur_year,cur_month,day_val)
                
                m1 = None
                m2 = None
                m3 = None
                m4 = None
                
                #----------------------------------------------------------
                # Check if DEP and ARR can both be on the same day
                # for consecutive bookings?

                # If its an unavailable date then mark: 
                m1 = re.search('c-calendar--unavailable', str(td))

                if m1 is not None:
                    #print("Found Unavailable..." + str(date_full))
                    all_months[str(date_full)] = 'UVL' # Unavailable day

                # If its an unavailable date then mark: 
                m2 = re.search('c-calendar--arrival-day', str(td))

                if m2 is not None:
                    print("Found Arrival..." + str(date_full))
                    all_months[str(date_full)] = 'ARR' # Arrival day

                # If its an unavailable date then mark: 
                m3 = re.search('c-calendar--departure-day', str(td))

                if m3 is not None:
                    #print("Found Departure... " + str(date_full))
                    all_months[str(date_full)] = 'DEP' # Departure day

                m4 = re.search('c-calendar--available', str(td))
                
                #if re.search('c-calendar--available', str(td)) is not None:
                    #print("Available*******************")
                
                if m4 is not None:
                    #print("Found available..."  + str(date_full))
                    all_months[str(date_full)] = 'AVL2' # Available day


            except ValueError:
                day_val = None

        # Move to the next month
        print("Moving to next month---------------------------")
        print(all_months)
        print("-----------------------------------------------")
        cur_month += 1
        month_tb_count += 1

    # Print the table to check
    #print(all_months)
    
    cal_details = {}
    cal_details['property_id'] = row['property_id']
    cal_details['calendar'] = all_months
    
    print(all_months)
    

    
    if first_page is True:
        fp.write('[\n')
        first_page = False
    else:
        fp.write('\n,')
        
    # Write the data to JSON
    json.dump(cal_details, fp)

# Close out the JSON format
fp.write('\n]')    

# Tidy up
fp.close()

Current month: 3
Month_tb: 1
Length of cells:238
Found Arrival...2018-03-02
Found Arrival...2018-03-06
Found Arrival...2018-03-17
Found Arrival...2018-03-23
Found Arrival...2018-03-30
Found Arrival...2018-03-14
Found Arrival...2018-03-23
Found Arrival...2018-03-07
Found Arrival...2018-03-18
Found Arrival...2018-03-04
Found Arrival...2018-03-08
Found Arrival...2018-03-03
Found Arrival...2018-03-09
Moving to next month---------------------------
{'2018-03-01': 'AVL2', '2018-03-02': 'AVL2', '2018-03-03': 'AVL2', '2018-03-04': 'AVL2', '2018-03-05': 'AVL2', '2018-03-06': 'AVL2', '2018-03-07': 'AVL2', '2018-03-08': 'AVL2', '2018-03-09': 'AVL2', '2018-03-10': 'AVL2', '2018-03-11': 'AVL2', '2018-03-12': 'AVL2', '2018-03-13': 'AVL2', '2018-03-14': 'AVL2', '2018-03-15': 'AVL2', '2018-03-16': 'AVL2', '2018-03-17': 'AVL2', '2018-03-18': 'AVL2', '2018-03-19': 'AVL2', '2018-03-20': 'AVL2', '2018-03-21': 'AVL2', '2018-03-22': 'AVL2', '2018-03-23': 'AVL2', '2018-03-24': 'AVL2', '2018-03-25': 'AVL2', '

In [15]:

print(date_str)
p = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar_' + date_str + '.json')
p.head()

2018-03-04


,calendar,property_id
0,"{'2018-03-01': 'AVL2', '2018-03-02': 'AVL2', '...",9064241


In [16]:
c = p['calendar']
print(c.head())

c2 = c.to_dict()
type(c2)


for key, value in c2[0].items() :
    print (key, value)


0    {'2018-03-01': 'AVL2', '2018-03-02': 'AVL2', '...
Name: calendar, dtype: object
2018-03-01 AVL2
2018-03-02 AVL2
2018-03-03 AVL2
2018-03-04 AVL2
2018-03-05 AVL2
2018-03-06 AVL2
2018-03-07 AVL2
2018-03-08 AVL2
2018-03-09 AVL2
2018-03-10 AVL2
2018-03-11 AVL2
2018-03-12 AVL2
2018-03-13 AVL2
2018-03-14 AVL2
2018-03-15 AVL2
2018-03-16 AVL2
2018-03-17 AVL2
2018-03-18 AVL2
2018-03-19 AVL2
2018-03-20 AVL2
2018-03-21 AVL2
2018-03-22 AVL2
2018-03-23 AVL2
2018-03-24 AVL2
2018-03-25 AVL2
2018-03-26 AVL2
2018-03-27 AVL2
2018-03-28 AVL2
2018-03-29 AVL2
2018-03-30 AVL2
2018-03-31 AVL2
2018-04-01 AVL2
2018-04-02 AVL2
2018-04-03 AVL2
2018-04-04 AVL2
2018-04-05 AVL2
2018-04-06 AVL2
2018-04-07 AVL2
2018-04-08 AVL2
2018-04-09 AVL2
2018-04-10 AVL2
2018-04-11 AVL2
2018-04-12 AVL2
2018-04-13 AVL2
2018-04-14 AVL2
2018-04-15 AVL2
2018-04-16 AVL2
2018-04-17 AVL2
2018-04-18 AVL2
2018-04-19 AVL2
2018-04-20 AVL2
2018-04-21 AVL2
2018-04-22 AVL2
2018-04-23 AVL2
2018-04-24 AVL2
2018-04-25 AVL2
2018-04-26 AVL2
201